In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
import xgboost

In [3]:
from sklearn.datasets import load_boston
data1=load_boston()
boston = pd.DataFrame(data1.data, columns=data1.feature_names)
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [4]:
boston["MEDV"] = data1.target
boston

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [5]:
Y= boston['MEDV']
X = pd.DataFrame(data1.data, columns=data1.feature_names)
X_train, X_test, Y_train, Y_test=train_test_split(X, Y, test_size=0.2, random_state=123)


In [14]:
tpot = TPOTRegressor(generations=20, population_size=50, verbosity=2)
tpot.fit(X_train, Y_train)
print(tpot.score(X_test, Y_test))
tpot.export('tpot_boston_pipeline.py')

Optimization Progress:   0%|          | 0/1050 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -10.269465457111014

Generation 2 - Current best internal CV score: -9.019667111299224

Generation 3 - Current best internal CV score: -9.019667111299224

Generation 4 - Current best internal CV score: -9.019667111299224

Generation 5 - Current best internal CV score: -9.019667111299224

Generation 6 - Current best internal CV score: -9.019667111299224

Generation 7 - Current best internal CV score: -9.019667111299224

Generation 8 - Current best internal CV score: -9.019667111299224

Generation 9 - Current best internal CV score: -9.019667111299224

Generation 10 - Current best internal CV score: -8.93430028801635

Generation 11 - Current best internal CV score: -8.830169880657802

Generation 12 - Current best internal CV score: -8.830169880657802

Generation 13 - Current best internal CV score: -8.830169880657802

Generation 14 - Current best internal CV score: -8.639016414031124

Generation 15 - Current best internal CV score: -8.63901

In [13]:
from xgboost import XGBRegressor
model = XGBRegressor(learning_rate=0.1, max_depth=10,
                     min_child_weight=13, n_estimators=100, n_jobs=1, 
                     subsample=0.8500000000000001, verbosity=0)
model.fit(X_train, Y_train,
          eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_test, Y_test)], 
    verbose=True)
print(model.score(X_test, Y_test))

[0]	validation_0-rmse:21.47948	validation_1-rmse:22.13353
[1]	validation_0-rmse:19.48013	validation_1-rmse:20.10718
[2]	validation_0-rmse:17.64775	validation_1-rmse:18.29743
[3]	validation_0-rmse:16.01735	validation_1-rmse:16.65063
[4]	validation_0-rmse:14.53314	validation_1-rmse:15.18013
[5]	validation_0-rmse:13.22217	validation_1-rmse:13.85044
[6]	validation_0-rmse:12.02004	validation_1-rmse:12.66792
[7]	validation_0-rmse:10.97037	validation_1-rmse:11.60563
[8]	validation_0-rmse:10.01704	validation_1-rmse:10.70900
[9]	validation_0-rmse:9.15307	validation_1-rmse:9.90025
[10]	validation_0-rmse:8.37999	validation_1-rmse:9.18856
[11]	validation_0-rmse:7.68208	validation_1-rmse:8.52029
[12]	validation_0-rmse:7.04194	validation_1-rmse:7.92400
[13]	validation_0-rmse:6.47433	validation_1-rmse:7.41035
[14]	validation_0-rmse:5.96272	validation_1-rmse:6.94247
[15]	validation_0-rmse:5.50657	validation_1-rmse:6.58707
[16]	validation_0-rmse:5.10054	validation_1-rmse:6.31245
[17]	validation_0-rmse:

In [18]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(alpha=0.99, learning_rate=0.1, max_depth=6, max_features=0.35000000000000003, 
                          min_samples_leaf=7,
                          min_samples_split=19, n_estimators=100, subsample=0.8500000000000001)
model.fit(X_train, Y_train)
print(model.score(X_test, Y_test))

0.8012060674470933
